* Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All). Or, alternatively, **Restart & Run All**.

* Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE".

* You can always add additional cells to the notebook to experiment, to test your answers, or to provide additional support for your answers.

* You should not need to install new packages to complete an assignment. If you use any packages not available via the provided `Project.toml`, or if you change the `Manifest.toml`, then your assignment will likely not be graded correctly.

* Submissions are only accepted via CANVAS!

* Late submissions will not be accepted and receive 0%. There will be no exceptions.

* By entering your name below you confirm that you have completed this assignment on your own. Suspicion of plagiarism / copying may result in an ad hoc oral examination on the content of the assignment. I may then downgrade an assignment. 

In [ ]:
STUDENTID = ""

---

# MATH 405/607 

# Numerical Methods for Differential Equations

## Assignment 5: Spectral Methods 

* This assignment will be worth 10% of the overall grade.

In [ ]:
include("math405.jl")

# Here are the codes you need to construct and evaluate trigonometric interpolants

using FFTW
"construct a real-space grid - use FFT convention"
xgrid(N) = range(0.0, stop=2π-2π/2N, length=2N)
"construct the reciprocal space grid assumed by `fft` - carefully use FFT convention"
kgrid(N) = [0:N; -N+1:-1]
"compute the fourier coefficients of I_N"
fouriercoeffs(f::Function, N) = fouriercoeffs(f.(xgrid(N)))
fouriercoeffs(F::Vector) = fft(F) / length(F)
"evaluate a trigonometric polynomial"
eval_trig(F::Function, N, Z) = eval_trig(fouriercoeffs(F, N), Z)
eval_trig(F̂::Vector, Z) = real(sum( f̂k * exp.((im * k)*Z) for (f̂k, k) in zip(F̂, kgrid(length(F̂)÷2)) ))

# To evaluate the trigonometric interpolant of a function F 
# at several points Zs::AbstractVector use the following function 
eval_trig(F::Function, N::Integer, Zs::AbstractVector) = eval_trig.( Ref(fouriercoeffs(F, N)), Zs )

<a name = 'q1'></a>
### Question 1: Approximation vs Regularity [15]

Let $f : \mathbb{R} \to \mathbb{R}$ be continuous and $2\pi$-periodic and let $I_N f$ denote its degree $N$ trigonometric interpolant.

In class we have shown that if $f$ is analytic and bounded, $|f| \leq M$, in the strip $\{ |{\rm Im} z| \leq \alpha \}$ then 
$$
  \| f - I_N f \|_\infty \lesssim (\log N) M e^{-\alpha N}.  
$$

Similarly, the **Jackson Theorem(s)** state that, if $f$ is $2\pi$-periodic, $p$ times continuously differentiable and the $p$th derivative $f^{(p)}$ is $s$-Hölder continuous, $s \in (0, 1]$, i.e.
$$
    |f^{(p)}(x) - f^{(p)}(y)| \leq C |x - y|^s
$$
then 
$$
    \|f - I_N f\|_\infty \lesssim (\log N) |N|^{-p-s}.
$$
The prototype of an  $s$-Hölder continuous function is $|x|^s$. A piecewise smooth function with finitely many $|x|^s$ singularities is also $s$-Hölder continuous in the sense defined above. The $\log N$ factor is a technicality, very difficult to see numerically (if it is indeed sharp), and should simply be ignored for this Question. 

Following the Model solution, produce figures demonstrating the algebraic convergence rates stated in the above estimates. As in the model solution (taken from class notes), in addition to plotting the convergence, also plot your predicted rate of convergence.

(i) $f(x) = |\sin(x)|$

(ii) $f(x) = |\sin(x)|^5$

(iii) $f(x) = |\sin(x)|^{7/3}$

In [ ]:
# Model Solution 
f = x -> 1 / (1+sin(x)^2)
NN = 4:2:16
xerr = range(0, 2*pi, length=10_000)
Err = [ norm(f.(xerr) - eval_trig(f, N, xerr), Inf) for N in NN ]
println("Rate for 1 / (1+sin(x)^2): error ~ exp(- 0.87 N)")
plot(NN, Err, lw=2, label = "1 / (1+sin(x)^2)", ms=5, m=:o, size = (500, 250), yscale = :log10, 
     xlabel = "N", ylabel = "error")
plot!(NN[5:end], exp.(- 0.87 * NN[5:end]), lw=2, c=:black, ls = :dash, label = "")


In [ ]:
# YOUR CODE HERE

<a name = 'q2'></a>



### Question 2: Periodic Trapezoidal Rule  [3 + 5 + 3 + 4]

Let $f : \mathbb{R} \to \mathbb{R}$ be continuous and $2\pi$-periodic, then we can represent it in terms of its Fourier series, 
$$ 
    f(x) = \sum_{k \in \mathbb{Z}} \hat{f}_k e^{i k x}.
$$
We are interested in approximating the integral 
$$
    I := \int_{-\pi}^\pi f(x) \,dx 
$$
by the composite trapezoidal rule 
$$
    I_N := \frac{2\pi}{2N} \sum_{n = -N+1}^N f\big(n\pi/N\big).
$$
In the following tasks you may interchange limits freely without rigorous justification, in particular summation and integration.

(a) Prove that $I = 2 \pi \hat{f}_0$.

(b) Prove that 
$$
    \sum_{n = -N+1}^N e^{i k \pi n / N} = 
    \begin{cases} 
        2N, & k \in 2N \mathbb{Z},  \\ 
        0, & \text{otherwise}.
    \end{cases}
$$
and use this to deduce that 
$$
    I_N = 2\pi \sum_{k \in 2N\mathbb{Z}} \hat{f}_k.
$$
*REMARK: $k \in 2 N \mathbb{Z}$ means that $k = 2 N m$ for some integer $m$.*


(c) Deduce from (a, b) that 
$$
    |I_N - I| \leq 2\pi \sum_{k \in 2N\mathbb{Z} \setminus \{0\}} |\hat{f}_k|.
$$

(d) Suppose that (i) $|\hat{f}_k| \leq M e^{-\alpha |k|}$; or (ii) $|\hat{f}_k| \leq C |k|^{-p}$. In each case deduce from (c)  a sharp (up to constants) estimate on the error $|I_N - I|$.

HINT: If $\eta(x)$ is non-negative and monotonically decreasing then $\sum_{k = K+1}^\infty \eta(k) \leq \int_{K}^\infty \eta(x) dx$.

YOUR ANSWER HERE

<a name = 'q3'></a>

### Question 3: Fourth-order BVP [5 + 5 + 5]

Consider the periodic boundary value problem for the differential equation 
$$
    u_{xxxx} + u = f 
$$

(a) Prove that, if $u$ solves the above DE, then 
$$ 
    \|u\|_{H^2} := ( \|u_{xx}\|_{L^2}^2+\|u\|_{L^2}^2 )^{1/2} \leq \|f\|_{L^2}.
$$
where $\|f\|_{L^2} := (\int_0^{2\pi} f^2 dx )^{1/2}$.  
*[HINT: multiply the DE by $u$, then integrate over $[0, 2\pi]$, integrate by parts, then use the Cauchy-Schwarz inequality on the right-hand side.]*

(b) Derive the canonical Fourier spectral method for this problem, providing both the real-space formulation as well as the fully discrete formulation in reciprocal space. Briefly specify all your terms.


(c) Suppose that $f$ is analytic and bounded in the strip $\{ |{\rm Im} z| \leq \alpha \}$, prove that 
$$
     \|u - u_N \|_{H^2} \leq C e^{-\alpha N}.
$$
where $u_N$ denotes the solution to the Fourier-spectral method from (a) and $u$ the exact solution of the DE.


YOUR ANSWER HERE

**BONUS [NO CREDIT]** 

Q3(d) Employing the method of manufactured solutions, confirm your prediction Question 3a-c.

In [ ]:
# YOUR CODE HERE

<a name = 'q5'></a>

### Question 4: Eigenvalue problem [10]

Find all eigenpairs $(\lambda, u)$, $\lambda \in \mathbb{C}$, $u$ smooth and $2\pi$-periodic, satisfying 
$$
0.1 u_{xxxx} + u_{xxx} = \lambda u_{xx},
$$
and such that $|\lambda| \leq 10$. Plot them in a scatter plot and store them (unordered) in a variable 
```julia
Lambda = ...
```
where `Lambda::AbstractVector` containing precisely the required set of eigenvalues.

*HINT:* The code required to complete this problem is only about 5 lines.

If you wish you may add any additional comments or notes in this box: 



In [ ]:
# YOUR CODE HERE